In [1]:
import pandas as pd
import MySQLdb
import nltk
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import jsonify

In [2]:
db = MySQLdb.connect("69.89.14.35","360feedback","360feedback@","ttn_central_v1" )
db.autocommit=False

In [3]:
df = pd.read_sql_query(sql="""SELECT 
    360_v1_respondent_feedback.*
FROM
    ttn_central_v1.360_v1_respondent_feedback
        JOIN
    360_v1_respondent_fs_rel_mapper ON 360_v1_respondent_fs_rel_mapper.RFRM_ID = 360_v1_respondent_feedback.RFRM_ID
WHERE
    360_v1_respondent_feedback.SSUBM_ID IS NOT NULL
        -- AND 360_v1_respondent_fs_rel_mapper.PROJ_ID = 525;""",con=db)

In [9]:
df

,RF_ID,CSM_ID,RATE,SSUBM_ID,SUBJECTIVE,RFRM_ID
0,25,None,0,4,Work life balance.,20
3,30,None,0,3,drhfj,6
6,81,None,0,4,Do a self assessment and then start his work,10
7,82,None,0,5,Be proactive and do things on time. Work for t...,10
8,107,None,0,4,1) Take steps to harmonize and communicate HR ...,11
10,133,None,0,4,Not Applicable,8
11,134,None,0,5,Not Applicable.,8
14,185,None,0,4,Work Life Balance,19
15,186,None,0,5,Getting personal on professional feedback rece...,19
16,188,None,0,3,efrggtrghrth,7


In [10]:
df = df[~df.SUBJECTIVE.isin(["nopes","","."])]

In [11]:
df["SUBJECTIVE"]=df.SUBJECTIVE.map(lambda x: x.lower())

In [12]:
df.head()

,RF_ID,CSM_ID,RATE,SSUBM_ID,SUBJECTIVE,RFRM_ID
0,25,None,0,4,work life balance.,20
3,30,None,0,3,drhfj,6
6,81,None,0,4,do a self assessment and then start his work,10
7,82,None,0,5,be proactive and do things on time. work for t...,10
8,107,None,0,4,1) take steps to harmonize and communicate hr ...,11


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
df['tokens']=df.SUBJECTIVE.apply(lambda x: nltk.wordpunct_tokenize(x))
df.tokens.head()

0                             [work, life, balance, .]
3                                              [drhfj]
6    [do, a, self, assessment, and, then, start, hi...
7    [be, proactive, and, do, things, on, time, ., ...
8    [1, ), take, steps, to, harmonize, and, commun...
Name: tokens, dtype: object

In [14]:
stemmer = SnowballStemmer("english")
df['stemmed'] = df.tokens.map(lambda x: " ".join([stemmer.stem(y) for y in x if y.isalpha()]))
df['stemmedarr'] = df.tokens.map(lambda x: [stemmer.stem(y) for y in x if y.isalpha()])
df.head()

,RF_ID,CSM_ID,RATE,SSUBM_ID,SUBJECTIVE,RFRM_ID,tokens,stemmed,stemmedarr
0,25,None,0,4,work life balance.,20,"[work, life, balance, .]",work life balanc,"[work, life, balanc]"
3,30,None,0,3,drhfj,6,[drhfj],drhfj,[drhfj]
6,81,None,0,4,do a self assessment and then start his work,10,"[do, a, self, assessment, and, then, start, hi...",do a self assess and then start his work,"[do, a, self, assess, and, then, start, his, w..."
7,82,None,0,5,be proactive and do things on time. work for t...,10,"[be, proactive, and, do, things, on, time, ., ...",be proactiv and do thing on time work for the ...,"[be, proactiv, and, do, thing, on, time, work,..."
8,107,None,0,4,1) take steps to harmonize and communicate hr ...,11,"[1, ), take, steps, to, harmonize, and, commun...",take step to harmon and communic hr process to...,"[take, step, to, harmon, and, communic, hr, pr..."


In [15]:
# Starting with the CountVectorizer/TfidfTransformer approach...
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
 
cvec = CountVectorizer(stop_words='english', min_df=1, max_df=.5, ngram_range=(1,2))
cvec

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [16]:
# Calculate all the n-grams found in all documents
from itertools import islice
cvec.fit(df.stemmed)
list(islice(cvec.vocabulary_.items(), 20))

[(u'vertic extern', 46375),
 (u'intern recognit', 21884),
 (u'skill provid', 39187),
 (u'work manag', 47578),
 (u'goal gmr', 17288),
 (u'speedi manner', 39956),
 (u'competit goal', 7448),
 (u'senior organis', 38003),
 (u'lot potenti', 25127),
 (u'build maintain', 5091),
 (u'work favour', 47466),
 (u'relast target', 35278),
 (u'feed junior', 15533),
 (u'latest practic', 23374),
 (u'time deliveri', 44055),
 (u'scold', 37599),
 (u'necess runner', 27694),
 (u'sinc supersed', 38814),
 (u'command like', 6879),
 (u'team set', 42970)]

In [17]:
# Check how many total n-grams we have
ngramrange=len(cvec.vocabulary_)

In [18]:
# Check how many total n-grams we have
len(cvec.vocabulary_)

48071

In [19]:
cvec_counts = cvec.transform(df.stemmed)
print 'sparse matrix shape:', cvec_counts.shape
print 'nonzero count:', cvec_counts.nnz
print 'sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1]))

sparse matrix shape: (8598, 48071)
nonzero count: 141093
sparsity: 0.03%


In [20]:
occ = np.asarray(cvec_counts.sum(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'term': cvec.get_feature_names(), 'occurrences': occ})
counts_df.sort_values(by='occurrences', ascending=False).head(20)

,occurrences,term
42525,2736,team
47272,1825,work
10189,909,deliv
43951,877,time
23713,851,learn
36160,850,respect
35363,825,relationship
20656,810,individu
36308,805,respons
33576,783,promis


In [21]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
transformed_weights

<8598x48071 sparse matrix of type '<type 'numpy.float64'>'
	with 141093 stored elements in Compressed Sparse Row format>

In [22]:
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
42525,team,0.025408
27603,na,0.022563
13625,entrepreneurship,0.022148
47272,work,0.020469
19452,humil,0.019485
10189,deliv,0.019236
33576,promis,0.018557
10255,deliv promis,0.018242
36160,respect,0.017428
20656,individu,0.016656


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(min_df=.0025, max_df=.1, stop_words='english')
tvec_weights = tvec.fit_transform(df.stemmed.dropna())
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(10)

,term,weight
139,deliv,0.034500
181,entrepreneurship,0.033878
254,humil,0.033124
402,promis,0.032749
432,respect,0.031375
425,relationship,0.031260
296,learn,0.029961
265,individu,0.029944
433,respons,0.028923
468,social,0.024750


In [24]:
weights_df.size

1096

In [25]:
top_weight=weights_df.sort_values(by='weight', ascending=False).head(50)
top_weight.head(5)

,term,weight
139,deliv,0.034500
181,entrepreneurship,0.033878
254,humil,0.033124
402,promis,0.032749
432,respect,0.031375


In [26]:
df["close"]=df.stemmedarr.map(lambda x: len([y for y in x if y in top_weight.term.values]))

In [27]:
df.sort_values(by="close",ascending=False).head()

,RF_ID,CSM_ID,RATE,SSUBM_ID,SUBJECTIVE,RFRM_ID,tokens,stemmed,stemmedarr,close
4602,69982,None,0,25,the positive you and the normal you are very d...,2532,"[the, positive, you, and, the, normal, you, ar...",the posit you and the normal you are veri diff...,"[the, posit, you, and, the, normal, you, are, ...",52
4601,69981,None,0,24,you have an excellent memory \r\n\r\nability t...,2532,"[you, have, an, excellent, memory, ability, to...",you have an excel memori abil to grasp thing i...,"[you, have, an, excel, memori, abil, to, grasp...",34
9555,157937,None,0,71,mr. govind very strongly demonstrates humility...,5803,"[mr, ., govind, very, strongly, demonstrates, ...",mr govind veri strong demonstr humil and learn...,"[mr, govind, veri, strong, demonstr, humil, an...",22
10010,168633,None,0,78,1) social responsibility: by volunteering in t...,6191,"[1, ), social, responsibility, :, by, voluntee...",social respons by volunt in the social activ a...,"[social, respons, by, volunt, in, the, social,...",20
10009,168632,None,0,77,1) deliver the promise: he always take full re...,6191,"[1, ), deliver, the, promise, :, he, always, t...",deliv the promis he alway take full respons of...,"[deliv, the, promis, he, alway, take, full, re...",20


In [28]:
[y for y in df.SUBJECTIVE.values]

['work life balance.',
 'drhfj',
 'do a self assessment and then start his work',
 'be proactive and do things on time. work for the organisation not for individual',
 '1) take steps to harmonize and communicate hr processes to all napierites\r\n2) develop the team to enable quicker tat for recruiting requests\r\n3) continue to communicate better',
 'not applicable',
 'not applicable.',
 'work life balance',
 'getting personal on professional feedback received',
 'efrggtrghrth',
 'though it is understood that the hr head position has quite of bit of urgent task lined up, it would be good if he can spend 15 mins daily  or 30 mins twice a week with the team to help complete the approval, revision process for pending tasks,',
 'give a very slight pause between some critical  decision made and it being communicated for execution, this is just to ensure all aspects, stakeholders affected by the decision are considered in totality before we actually execute it.',
 'dfg',
 'df',
 'yes, defini